In [1]:
from os import environ

environ.setdefault("JAVA_HOME", "/Library/Java/JavaVirtualMachines/openjdk-11.jdk/Contents/Home/")

import groovy

In [2]:
from importlib import reload
from geo import *
import aiohttp
import noise

pt = Point(55.771820853060554,37.57251262664795, ANGLE)

In [3]:
sess = aiohttp.ClientSession()

comp = noise.NoiseComputation(pt, sess)

print(noise.overpass_query(comp.pt, noise.AREA_RADIUS))

await comp.fetch_map()

(
        way[building](around:1000,55.771820853060554,37.57251262664795);
way[highway](around:1000,55.771820853060554,37.57251262664795);
    );
    out meta;
    >;
    out meta qt;

got  51980
got  113662
got  165948
got  289629
got  359104
got  422610
got  551439
got  665162
got  726023
got  795513
got  868808
got  1006400
got  1087998
got  1151371
got  1293089
got  1435866
got  1506245
got  1584097
got  1746312
got  1905782
got  1982642
got  2151376
got  2389717
got  2559159
got  2642969
got  2820460
got  2890954
got  2968420
got  3054639
got  3500212
got  3572597
got  3780587
got  3844392
got  4083030


In [4]:
await comp.import_osm()

[Thread-1] INFO org.noise_planet.noisemodelling - Start : Get Buildings from OSM
[Thread-1] INFO org.noise_planet.noisemodelling - inputs {ignoreGround=true, removeTunnels=true, targetSRID=3857, pathFile=/tmp/osm_16925581214162700000.osm, cid=16925581214162700000}
[Thread-1] INFO org.noise_planet.noisemodelling - OSM Read done
[Thread-1] WARN org.cts.CRSHelper - A grid has been found.
[Thread-1] INFO org.noise_planet.noisemodelling - SQL INSERT done
[Thread-1] INFO org.noise_planet.noisemodelling - End : Get Buildings from OSM
[Thread-1] INFO org.noise_planet.noisemodelling - Result : nodes : 15376<br>
ways : 3376<br>
relations : 0<br>
buildings : 799<br>
roads : 2522<br>
grounds : 0<br>



In [5]:
import db
zz = await db.query_all(await comp.db_conn(), "select * from buildings limit 1")
zz[0]['THE_GEOM'].getSRID()

3857

In [6]:
import db
reload(db)
reload(noise)
bld = await noise.NoiseComputation.get_building(comp)
ss = await db.query_all(await comp.db_conn(), f"select min(ST_Length(ST_ShortestLine(?, the_geom))) as min_dist from {noise.BUILDINGS_TABLE}", [comp.db_pt])
zz = await db.query_all(await comp.db_conn(), f"select * from {noise.BUILDINGS_TABLE} where ST_Length(ST_ShortestLine(?, the_geom)) <= 0 limit 1", [comp.db_pt])
print("ss", ss)
print("zz", zz)
print(bld)
print(comp.db_pt)

ss [{'MIN_DIST': 0.0}]
zz [{'PK': 661016854, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Polygon'>, 'HEIGHT': 48.03764724731445}]
POLYGON ((4182523.5821102858 7513135.988861043, 4182514.7433427167 7513137.0377580635, 4182513.285057388 7513125.678012989, 4182490.019283812 7513128.686166853, 4182491.3217218537 7513138.8782758275, 4182486.9357339167 7513139.452200814, 4182488.5387345836 7513151.9004463125, 4182492.9247225216 7513151.346310946, 4182493.7596187023 7513157.778241708, 4182515.7897459306 7513154.948191513, 4182517.092183973 7513165.080963437, 4182578.206584418 7513157.224105926, 4182564.5588148464 7513050.9693049025, 4182537.252143755 7513054.472187103, 4182547.8497592784 7513136.919015001, 4182537.4525188385 7513138.264769856, 4182536.9293172327 7513134.267086994, 4182523.5821102858 7513135.988861043))
POINT (4182552.973422291 7513125.094799487)


In [7]:
await noise.NoiseComputation.create_receivers_for_building(comp)

invoke building_grid: POLYGON ((4182470.110830952 7513128.81729, 4182469.0165734612 7513130.56942599, 4182467.629276716 7513134.231021794, 4182466.9819904948 7513138.092743771, 4182467.0995250773 7513142.006573316, 4182468.702525744 7513154.454818814, 4182469.5801008907 7513158.269921054, 4182471.1840354796 7513161.840990513, 4182473.4528781925 7513165.031209311, 4182476.299703181 7513167.718351051, 4182476.4840073544 7513167.834029016, 4182478.6923797843 7513170.930363643, 4182481.545130678 7513173.615106369, 4182484.866491357 7513175.692266549, 4182488.5290375594 7513177.08215377, 4182492.392255415 7513177.731444901, 4182496.3079322698 7513177.615229835, 4182501.141191903 7513176.994336127, 4182502.0151706985 7513178.221879339, 4182504.868580882 7513180.910793742, 4182508.19175056 7513182.9913594425, 4182511.8569685747 7513184.383619206, 4182515.72337867 7513185.034067758, 4182519.6423926726 7513184.917708019, 4182580.7567931176 7513177.060850508, 4182584.577477923 7513176.182262903,

[Thread-1] INFO org.noise_planet.noisemodelling - Start : Receivers grid around buildings
[Thread-1] INFO org.noise_planet.noisemodelling - inputs {tableBuilding=BUILDINGS, sourcesTableName=ROADS, fence=POLYGON ((4182470.110830952 7513128.81729, 4182469.0165734612 7513130.56942599, 4182467.629276716 7513134.231021794, 4182466.9819904948 7513138.092743771, 4182467.0995250773 7513142.006573316, 4182468.702525744 7513154.454818814, 4182469.5801008907 7513158.269921054, 4182471.1840354796 7513161.840990513, 4182473.4528781925 7513165.031209311, 4182476.299703181 7513167.718351051, 4182476.4840073544 7513167.834029016, 4182478.6923797843 7513170.930363643, 4182481.545130678 7513173.615106369, 4182484.866491357 7513175.692266549, 4182488.5290375594 7513177.08215377, 4182492.392255415 7513177.731444901, 4182496.3079322698 7513177.615229835, 4182501.141191903 7513176.994336127, 4182502.0151706985 7513178.221879339, 4182504.868580882 7513180.910793742, 4182508.19175056 7513182.9913594425, 41825

In [8]:
r = await db.query_all(await comp.db_conn(), "select * from receivers limit 20")
sz = await db.query_all(await comp.db_conn(), "select count(*) from receivers")
print(r)
print(sz)

print("\n".join(str(x['THE_GEOM'].toString()) for x in r))

[{'PK': 1, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 2, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 3, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 4, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 5, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 6, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 7, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 8, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 9, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 10, 'THE_GEOM': <java object 'org.locationtech.jts.geom.Point'>, 'BUILD_PK': 661016854}, {'PK': 11, 'THE_GEOM': <java object 'org.locatio

In [9]:
await comp.compute_road_emision()

[Thread-1] INFO org.noise_planet.noisemodelling - Start : Road Emission from DEN
[Thread-1] INFO org.noise_planet.noisemodelling - inputs {tableRoads=ROADS}
[Thread-1] INFO org.noise_planet.noisemodelling - The table Roads has 2522 road segments.
[Thread-1] INFO org.noise_planet.noisemodelling - 
Result : Calculation Done ! The table LW_ROADS has been created.
[Thread-1] INFO org.noise_planet.noisemodelling - End : LW_ROADS from Emission


In [10]:
await comp.compute_propagation()

[Thread-1] INFO org.noise_planet.noisemodelling - Start : LDEN from Emission
[Thread-1] INFO org.noise_planet.noisemodelling - inputs {tableReceivers=RECEIVERS, tableBuilding=BUILDINGS, tableSources=LW_ROADS}
[Thread-1] INFO org.noise_planet.noisemodelling - Start calculation... 
[Thread-1] INFO org.noise_planet.noisemodelling.jdbc.PointNoiseMap - Collect all receivers in order to localize populated cells
[Thread-1] INFO org.noise_planet.noisemodelling - Compute domain is POLYGON ((4182224.9125 7513111.5125, 4182224.9125 7513412.4, 4182500.7 7513412.4, 4182500.7 7513111.5125, 4182224.9125 7513111.5125))
[Thread-1] INFO org.noise_planet.noisemodelling - Compute... 0.000 % (6 receivers in this cell)
[Thread-1] INFO org.noise_planet.noisemodelling.jdbc.PointNoiseMap - Begin processing of cell 120 / 256
[Thread-1] INFO org.noise_planet.noisemodelling.jdbc.PointNoiseMap - This computation area contains 6 receivers 77 sound sources and 31 buildings
[pool-1-thread-3] INFO org.noise_planet.noi

In [11]:
await comp.get_result()

NoiseData(laeq_day=Decimal('58.84'), laeq_evening=Decimal('57.72'), laeq_night=Decimal('50.52'), laeq_den=Decimal('60.37'))